In [2]:
import numpy as np
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta
import os

In [3]:
wanted_features = ['Revenue','Gross Profit','Operating Income','Pretax Income','Diluted EPS','EBITDA',
           'ROA','ROE','Cash from Operations','Cash from Investing Activities',
           'Cash from Financing Activities','Total Current Assets','Total Long-Term Assets','Total Assets',
           'Total Current Liabilities','Total Noncurrent Liabilities','Total Liabilities','Total Equity']

1. Непонятные буковки это копия XOG.xlsx
2. CRBO есть в oil_gas_train, но нет в oil_gas_train_test, поэтому обойдемся без него

In [10]:
dir_path1 = 'data/raw/oil_gas_train_test'
dir_path2 = 'data/raw/oil_gas_train'

In [11]:
set(os.listdir(dir_path2)) - set(os.listdir(dir_path1)) 

{'CRBO.xlsx', '¦Ъ¦-¦¬¦¦¦-8.xlsx'}

In [12]:
set(os.listdir(dir_path1)) - set(os.listdir(dir_path2)) 

set()

In [13]:
dates = [date(2011, 3, 31) + relativedelta(months=i) for i in range(0, 124, 3)]

In [14]:
lst = []
for path in os.listdir(dir_path1):
    if 'CRBO' in path in path:
        continue
    else:
        data = pd.read_excel(f'{dir_path1}/{path}').iloc[27:, 8:]
        data = data[data.iloc[:, 0].isin(wanted_features)]
        data = data.transpose()
        header = data.iloc[0]
        data = data[1:]
        data.columns = header
        data = data.loc[:,~data.columns.duplicated()]
        data['Date'] = pd.to_datetime(dates)
        data['Company'] = path[:-5]
        data.reset_index(drop=True, inplace=True)
        lst.append(data)
dataframe = pd.concat(lst, ignore_index=True)

По показателями на момент t нужно предсказать Revenue и Operation income на момент t+1. Ключевые признаки нужно сдвинуть на dt=1 назад. Тогда получатся данные для задачи обучения с учителем.

In [15]:
dataframe['Revenue target'] = dataframe.groupby('Company')[['Revenue']].shift(-1)
dataframe['Operating Income target'] = dataframe.groupby('Company')[['Operating Income']].shift(-1)

dataframe.dropna(axis=0, inplace=True)

Из-за данного смещения, даты на который приходится тестовые данные также смещается на 1 квартал (test начинается с 2020-06-30 включительно)

![jupyter](res/pic1.png)

Перед сохранением избавимся от исходных revenue и operation revenue

In [47]:
#dataframe.drop(columns=['Revenue','Operating Income'], inplace = True)

Делим датафрейм на test и train

In [17]:
dataframe.to_csv('data/processed/data.csv', index=False)